# From BoW to tf-idf

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt
from nlpia.loaders import clean_columns
from nltk.tokenize import TreebankWordTokenizer

import copy
import pandas as pd

## The difference between a binary and a _counting_ BoW

In [ ]:
sentence = "The faster Harry got to the store, the faster Harry, the faster, would get home."
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())

tokens

In [ ]:
# Getting the vocabulary 
set(tokens)

In [ ]:
# Counting the tokens in the sentence
bag_of_words = Counter(tokens)
bag_of_words

In [ ]:
bag_of_words.most_common(4)

Getting the tf for a specific word in a document

In [ ]:
times_harry_appears = bag_of_words['harry']
num_unique_words = len(bag_of_words)         # what is this doing?

print("Times harry appears:", times_harry_appears)
print("Size of the vocabulary:", num_unique_words)

Back to the slides

### Normalising $tf$: dividing by the total number of words

In [ ]:
tf = times_harry_appears / num_unique_words
tf

**Note**: I have usually seen 

$\sum_w freq(w)$ 

as denominator. That is, sum of all the frequencies. 
I'm sticking to the book, which uses length of the vocabulary instead.

Back to the slides

## $tf$ on a slightly longer text

In [ ]:
# Wikipedia article on COVID as (roughly) one year ago
txt = """The Russo-Ukrainian War is an ongoing war primarily 
involving Russia, pro-Russian forces, and Belarus on one side, 
and Ukraine and its international supporters on the other. 
Conflict began in February 2014 following the Revolution of 
Dignity, and focused on the status of Crimea and parts of the 
Donbas, internationally recognised as part of Ukraine. The 
conflict includes the Russian annexation of Crimea (2014), 
the war in Donbas (2014–present), naval incidents, cyberwarfare, 
and political tensions. Intentionally concealing its involvement, 
Russia gave military backing to separatists in the Donbas from 
2014 onwards. Having built up a large military presence on the 
border from late 2021, Russia launched a full-scale invasion of 
Ukraine on 24 February 2022, which is ongoing.

Following the Euromaidan protests and a revolution resulting in 
the removal of pro-Russian President Viktor Yanukovych on 22 
February 2014, pro-Russian unrest erupted in parts of Ukraine. 
Russian soldiers without insignia took control of strategic 
positions and infrastructure in the Ukrainian territory of 
Crimea. Unmarked Russian troops seized the Crimean Parliament 
and Russia organized a widely-criticised referendum, whose 
outcome was for Crimea to join Russia. It then annexed Crimea. 
In April 2014, demonstrations by pro-Russian groups in the 
Donbas region of Ukraine escalated into a war between the 
Ukrainian military and Russian-backed separatists of the 
self-declared Donetsk and Luhansk republics.

In August 2014, unmarked Russian military vehicles crossed the 
border into the Donetsk republic. An undeclared war began between 
Ukrainian forces and separatists intermingled with Russian troops, 
although Russia denied the presence of its troops in the Donbas. 
The war settled into a stalemate, with repeated failed attempts 
at ceasefire. In 2015, a package of agreements called Minsk II 
were signed by Russia and Ukraine, but a number of disputes 
prevented them from being fully implemented. By 2019, 7% of 
Ukraine's territory was classified by the Ukrainian government 
as temporarily occupied territories, while the Russian government 
had indirectly acknowledged the presence of its troops in Ukraine.

In 2021 and early 2022, there was a major Russian military 
build-up around Ukraine's borders. NATO accused Russia of planning 
an invasion, which it denied. Russian President Vladimir Putin 
criticized the enlargement of NATO as a threat to his country and 
demanded Ukraine be barred from ever joining the military 
alliance. He also expressed Russian irredentist views, questioned 
Ukraine's right to exist, and stated Ukraine was wrongfully 
created by Soviet Russia. On 21 February 2022, Russia officially 
recognised the two self-proclaimed separatist states in the Donbas, 
and sent troops to the territories. Three days later, Russia 
invaded Ukraine after Putin announced a "special military 
operation". Much of the international community and organizations 
such as Amnesty International have condemned Russia for its actions 
in post-revolutionary Ukraine, accusing it of breaking 
international law and violating Ukrainian sovereignty. Many 
countries implemented economic sanctions against Russia, Russian 
individuals, or companies, especially after the 2022 invasion. 
"""

kk = """Coronavirus disease 2019 (COVID-19) is an infectious disease caused by 
severe acute respiratory syndrome coronavirus 2 (SARS coronavirus 2, 
or SARS-CoV-2), a virus closely related to the SARS virus. The disease 
was discovered and named during the 2019–20 coronavirus outbreak. 
Those affected may develop a fever, dry cough, fatigue, and shortness 
of breath. A sore throat, runny nose or sneezing is less common. While 
the majority of cases result in mild symptoms, some can progress to 
pneumonia and multi-organ failure.

The infection is spread from one person to others via respiratory droplets 
produced from the airways, often during coughing or sneezing. Time from exposure 
to onset of symptoms is generally between 2 and 14 days, with an average of 5 
days. The standard method of diagnosis is by reverse transcription polymerase 
chain reaction (rRT-PCR) from a nasopharyngeal swab or sputum sample, with 
results within a few hours to 2 days. Antibody assays can also be used, using a 
blood serum sample, with results within a few days. The infection can also be 
diagnosed from a combination of symptoms, risk factors and a chest CT scan 
showing features of pneumonia.

Correct handwashing technique, maintaining distance from people who are coughing 
and not touching one's face with unwashed hands are measures recommended to 
prevent the disease. It is also recommended to cover one's nose and mouth with a 
tissue or a bent elbow when coughing. Masks are also recommended for those who 
are taking care of someone with a suspected infection but not for the general 
public. There is no vaccine or specific antiviral treatment, with management 
involving treatment of symptoms, supportive care and experimental measures.
The case fatality rate is estimated at between 1% and 3%.

The World Health Organization (WHO) has declared the 2019–20 coronavirus 
outbreak a Public Health Emergency of International Concern (PHEIC). As of 7 
March 2020, evidence of local transmission of the disease has been found in 
multiple countries across all six WHO regions.
"""

In [ ]:
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(txt.lower())
token_counts = Counter(tokens)
# display the most common tokens in the document
token_counts.most_common(10)

The most frequent words are (almost) all **stopwords**!

In [ ]:
# Run only the first time
import nltk
nltk.download('stopwords', quiet=True)

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
# pay attention to what is going on here (do we need to unfold it?)
tokens = [x for x in tokens if x not in stopwords]
counts = Counter(tokens)
counts.most_common(10)

**Homework**: normalise these frequencies

back to the slides

## Vectorising

Not a **dictionary** of counts, but a **vector** of counts

In [ ]:
document_vector = []
doc_length = len(tokens)
for key, value in counts.most_common():
    document_vector.append(value / doc_length)
    
document_vector

In [ ]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")
print(docs)
print()
print("\n".join(docs))

In [ ]:
# Getting the full lexicon
doc_tokens = []
for doc in docs:
    #doc_tokens += [sorted(tokenizer.tokenize(doc.lower()))]
    doc_tokens.append(sorted(tokenizer.tokenize(doc.lower())))

print(len(doc_tokens[0]))
doc_tokens

In [ ]:
# concatenating all lists into one
all_doc_tokens = sum(doc_tokens, [])
print(len(all_doc_tokens))
all_doc_tokens

In [ ]:
lexicon = sorted(set(all_doc_tokens))
print(len(lexicon))

# Notice that usually the lexicon is called the set of types
lexicon

So, it is clear now: our vectors must have 18 values

Creating the initial zero vectors

In [ ]:
from collections import OrderedDict

# Creating a template dictionary with all zeros ("base vector")
zero_vector = OrderedDict((token, 0) for token in lexicon)
zero_vector

In [ ]:
# 1. Make copies of that base vector
# 2. Update the values of the vector for each document
# 3. Store them in an array
#
# We use copy.copy to avoid reference copies and do independent copies

doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    for key, value in token_counts.items():
        vec[key] = value / len(lexicon)
    doc_vectors.append(vec)
    
doc_vectors

Back to the slides

## Cosine similarity to compare two vectors

In [ ]:
import math

def cosine_sim(vec1, vec2):
    """
    cosine_sim computes the cosine similarity between two vectors
    :param vec1: Dictionary with vector (Counter)
    :param vec2: Dictionary with vector (Counter)
    :return: cosine(vec1, vec2)
    """
    
    #Convert our dictionaries into lists for easier matching
    vec1 = [val for val in vec1.values()]
    vec2 = [val for val in vec2.values()]

    dot_prod = 0
    for i, v in enumerate(vec1):
        dot_prod += v * vec2[i]

    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    # this parenthesis is important!
    return dot_prod / (mag_1 * mag_2)

In [ ]:
print("cosine(0, 1)=", cosine_sim(doc_vectors[0], doc_vectors[1]))
print("cosine(0, 2)=", cosine_sim(doc_vectors[0], doc_vectors[2]))
print("cosine(1, 2)=", cosine_sim(doc_vectors[1], doc_vectors[2]))
print("cosine(1, 1)=", cosine_sim(doc_vectors[1], doc_vectors[1]))

back to the slides

## Zipf's Law

Let us play with the Brown corpus to see Zipf's law. The [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) is one of the classical corpora in English, with 1M+ words, and a good alternative for testing.

In [ ]:
# Run only the first time
nltk.download('brown')

In [ ]:
from nltk.corpus import brown
# tokens
brown.words()[:10]

In [ ]:
# Including part of speech
brown.tagged_words()[:5]

In [ ]:
# Size of the Brown corpus (number of tokens)
len(brown.words())

In [ ]:
# We define a set of punctuation marks to filter them out
puncts = set((',', '.', '--', '-', '!', '?', ':', ';', '``', "''", '(', ')', '[', ']'))

word_list = (x.lower() for x in brown.words() if x not in puncts)
token_counts = Counter(word_list)
token_counts.most_common(20)

#Does this follow Zipf's distribution?

Back to the slides

## Inverse Document Frequency

In [ ]:
# Loading two sections of document "kite" (from NLP in Action)
from nlpia.data.loaders import kite_text, kite_history

kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)

kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)

print(intro_tokens)
print(history_tokens)

In [ ]:
intro_total = len(intro_tokens)
intro_total

In [ ]:
history_total = len(history_tokens)
history_total

In [ ]:
intro_tf = {}
history_tf = {}

intro_counts = Counter(intro_tokens)
intro_tf['kite'] = intro_counts['kite'] / intro_total

history_counts = Counter(history_tokens)
history_tf['kite'] = history_counts['kite'] / history_total

'Term Frequency of "kite" in intro is: {:.4f}'.format(intro_tf['kite'])

In [ ]:
'Term Frequency of "kite" in history is: {:.4f}'.format(history_tf['kite'])

$freq(kite, intro) \sim 2 * freq(kite, history)$

Is intro more about kites than history?

In [ ]:
# giving perspective by looking at the frequency of other words
intro_tf['and'] = intro_counts['and'] / intro_total
history_tf['and'] = history_counts['and'] / history_total
print('Term Frequency of "and" in intro is: {:.4f}'.format(intro_tf['and']))
print('Term Frequency of "and" in history is: {:.4f}'.format(history_tf['and']))

$freq(and, \cdot)$ is quite similar to $freq(kite, \cdot)$

So, the document is about **kites** and about **ands**?

(back to the slides)

In [ ]:
# Number of documents with *
num_docs_containing_and = 0
for doc in [intro_tokens, history_tokens]:
    if 'and' in doc:
        num_docs_containing_and += 1
        
num_docs_containing_kite = 0
for doc in [intro_tokens, history_tokens]:
    if 'kite' in doc:
        num_docs_containing_kite += 1
        
num_docs_containing_china = 0
for doc in [intro_tokens, history_tokens]:
    if 'china' in doc:
        num_docs_containing_china += 1
        
print("and:", num_docs_containing_and)
print("kite:", num_docs_containing_kite)
print("china:", num_docs_containing_china)

In [ ]:
# tf(china)
intro_tf['china'] = intro_counts['china'] / intro_total
history_tf['china'] = history_counts['china'] / history_total

print(intro_tf)
print(history_tf)

In [ ]:
# idf for all 3
num_docs = 2
idf = {}
idf['and'] = num_docs / num_docs_containing_and
idf['kite'] = num_docs / num_docs_containing_kite
idf['china'] = num_docs / num_docs_containing_china

print(idf)

In [ ]:
# tfidf for the intro

intro_tfidf = {}
intro_tfidf['and'] = intro_tf['and'] * idf['and']
intro_tfidf['kite'] = intro_tf['kite'] * idf['kite']
intro_tfidf['china'] = intro_tf['china'] * idf['china']

In [ ]:
# tfidf for the history
history_tfidf = {}
history_tfidf['and'] = history_tf['and'] * idf['and']
history_tfidf['kite'] = history_tf['kite'] * idf['kite']
history_tfidf['china'] = history_tf['china'] * idf['china']

In [ ]:
print(intro_tfidf)
print(history_tfidf)

Back to the slides

## Searching on "Harry"

In [ ]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")

In [ ]:
# not an extremely efficient implementation
document_tfidf_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    
    for key, value in token_counts.items():
        docs_containing_key = 0
        for _doc in docs:
            if key in _doc:
                docs_containing_key += 1
        tf = value / len(lexicon)
        if docs_containing_key:
            idf = len(docs) / docs_containing_key
        else:
            idf = 0
        vec[key] = tf * idf
    document_tfidf_vectors.append(vec)

document_tfidf_vectors
# Notice what happened to Harry!!

In [ ]:
query = "How long does it take to get to the hairy store?"
query_vec = copy.copy(zero_vector)
# query_vec = copy.copy(zero_vector)

tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)
 
for key, value in token_counts.items():
    docs_containing_key = 0
    for _doc in docs:
      if key in _doc.lower():
        docs_containing_key += 1
    if docs_containing_key == 0:
        continue
    tf = value / len(tokens)
    idf = len(docs) / docs_containing_key
    query_vec[key] = tf * idf
query_vec

In [ ]:
cosine_sim(query_vec, document_tfidf_vectors[0])

In [ ]:
cosine_sim(query_vec, document_tfidf_vectors[1])

In [ ]:
cosine_sim(query_vec, document_tfidf_vectors[2])

In [ ]:
print(docs)
print(query)

back to the slides

## Using sklearn to build tf-idf matrices

Now, rather than implementing everything ourselves, we will use a well-known python library to compute it for us.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Same small corpus as before
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")
corpus = docs

In [ ]:
vectorizer = TfidfVectorizer(min_df=1)
"""min_df: ignore terms that have a document frequency 
strictly lower than the given threshold (aka cut-off).
"""

model = vectorizer.fit_transform(corpus)
"""model is a sparse tf-idf matrix (mostly zeros) 
sklearn does not store zeros to save resources"""

print(model)

In [ ]:
# We can convert it into a matrix in one line!
print("\n--\n".join(corpus))
print(model.todense().round(2))

## End of the notebook